# Imports 

In [13]:
import pandas as pd
import re 
import numpy as np


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load and view dataset 1: tax certificates for San Diego businesses

In [14]:
## general link: https://data.sandiego.gov/datasets/business-listings/

## active tax certificates
sd_active_biz = pd.read_csv("https://seshat.datasd.org/ttcs/sd_businesses_active_datasd.csv")


## Step 1 - what are the possible join fields?

- Business-level fields:
    - Owner name
    - Business name (dba_name)
    
- Sector-level fields:
    - naics_sector 
    - naics_code
    - naics_description
    
- Geographic fields:
    - City and state (less interesting in this case)
    - Zip 
    - Bid (business improvement district)
    - Council district

## Step 2- once we've decided on join field, cleaning/deduplicating

Here, first focus on two-digit NAICS codes, or NAICS sector

In [15]:
## see reasonably clean and no missingness 
sd_active_biz.naics_sector.value_counts(dropna = False)

## look at crosstab with sector description
pd.crosstab(sd_active_biz.naics_sector,
           sd_active_biz.naics_description).T

54    13361
81     9588
56     5004
45     4940
62     4877
23     4830
44     3683
72     3600
53     3189
42     2049
48     1944
61     1790
71     1469
52     1219
33      845
51      803
32      325
49      315
31      309
55      234
11      116
22       68
92       11
21        2
Name: naics_sector, dtype: int64

naics_sector,11,21,22,23,31,32,33,42,44,45,...,53,54,55,56,61,62,71,72,81,92
naics_description,,,,,,,,,,,,,,,,,,,,,
ACCOMMODATION,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10,0,0
ACCOMMODATION & FOOD SERVICES,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,29,0,0
ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,0,0,0,0,0,0,0,0,0,0,...,0,529,0,0,0,0,0,0,0,0
ACTIVITIES RELATED TO CREDIT INTERMEDIATION,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACTIVITIES RELATED TO REAL ESTATE,0,0,0,0,0,0,0,0,0,0,...,297,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOOD PRODUCT MFG,0,0,0,0,0,23,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WRECKING & DEMOLITION CONTRACTORS,0,0,0,17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YOGA INSTRUCTOR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,80,0,0,0,0,0


## Step 3 - repeat those steps with the data i'm joining

### Here, I want to join official census data on naics codes that might have more detail, loading and cleaning


In [16]:
## general site- https://www.census.gov/eos/www/naics/2017NAICS

## two files that vary based on summary digits
naics_26 = pd.read_excel("https://www.census.gov/eos/www/naics/2017NAICS/2-6%20digit_2017_Codes.xlsx")
naics_26.head()
naics_6 = pd.read_excel("https://www.census.gov/eos/www/naics/2017NAICS/6-digit_2017_Codes.xlsx")
naics_6.head()

## see that requires two fixes to make usable:
## (1) column names
## (2) first row is just empty 
## (3) cols with unnamed are empty

def clean_naics(one_naics: pd.DataFrame):
    
    ## first fix cols
    naics_newcol = [re.sub('\s+|\.', '', col.lower()) for col in one_naics.columns]
    one_naics.columns = naics_newcol
    
    ## skip first row (0 index) and keep col if not unnamed in col
    naics_keep = one_naics.loc[1:, [col for col in one_naics.columns if "unnamed" not in col and "seq" not in col]]
    return(naics_keep)

,Seq. No.,2017 NAICS US Code,2017 NAICS US Title,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,11,"Agriculture, Forestry, Fishing and Hunting",NaN,NaN,NaN
2,2.0,111,Crop Production,NaN,NaN,NaN
3,3.0,1111,Oilseed and Grain Farming,NaN,NaN,NaN
4,4.0,11111,Soybean Farming,NaN,NaN,NaN


,2017 NAICS Code,2017 NAICS Title,Unnamed: 2
0,NaN,NaN,NaN
1,111110.0,Soybean Farming,NaN
2,111120.0,Oilseed (except Soybean) Farming,NaN
3,111130.0,Dry Pea and Bean Farming,NaN
4,111140.0,Wheat Farming,NaN


In [17]:
naics_26_c = clean_naics(naics_26)
naics_6_c = clean_naics(naics_6)

naics_26_c.head()
naics_6_c.head()

## rowbind using pd.concat and deduplicate
## to rowbind, need (1) identical colnames
## (2) good to also have identical types
print("Dtypes for 2-6 code data are:---------------------------")
naics_26_c.dtypes
print("Dtypes for 6 code data are:---------------------------")
naics_6_c.dtypes


## fix dtypes and rename col
naics_6_c['2017naicscode'] = naics_6_c['2017naicscode'].astype(str).str.replace("\.", "", regex = True)


## use regular expressions to rename cols by subbing out the "us"
### way 1: riskier since could have "us" elsewhere in string
naics_26_cleancol = [re.sub("us", "", one_col) for one_col in naics_26_c.columns]
naics_26_cleancol


,2017naicsuscode,2017naicsustitle
1,11,"Agriculture, Forestry, Fishing and Hunting"
2,111,Crop Production
3,1111,Oilseed and Grain Farming
4,11111,Soybean Farming
5,111110,Soybean Farming


,2017naicscode,2017naicstitle
1,111110.0,Soybean Farming
2,111120.0,Oilseed (except Soybean) Farming
3,111130.0,Dry Pea and Bean Farming
4,111140.0,Wheat Farming
5,111150.0,Corn Farming


Dtypes for 2-6 code data are:---------------------------


2017naicsuscode     object
2017naicsustitle    object
dtype: object

Dtypes for 6 code data are:---------------------------


2017naicscode     float64
2017naicstitle     object
dtype: object

['2017naicscode', '2017naicstitle']

In [18]:
### way 2: will make sense after datacamp module; tell it where to look
### for the "us" substring and remove that group
find_us_pattern = r"(.*naics)(us)([code|title].*)"

## let's test with 1 in a repetitive way (usually want to do
## the group call within an ifelse since issues if no matches)
found_g1 = re.match(find_us_pattern, naics_26_c.columns[0]).group(1)
found_g2 = re.match(find_us_pattern, naics_26_c.columns[0]).group(2)
found_g3 = re.match(find_us_pattern, naics_26_c.columns[0]).group(3)

print(found_g1)
print(found_g2)
print(found_g3)

## consolidate into one
cleaned_colpattern = [re.match(find_us_pattern, one_col).group(1) + re.match(find_us_pattern, one_col).group(3) 
                    for one_col
                    in naics_26_c.columns]

naics_26_c.columns = cleaned_colpattern

2017naics
us
code


In [19]:
## finally :) 
## we can concatenate!

naics_all_raw = pd.concat([naics_26_c, naics_6_c])
naics_all_raw.shape
naics_all = naics_all_raw.drop_duplicates()

naics_all['is_twodig_code'] = np.where(naics_all['2017naicscode'].astype(str).str.len() == 2, 
                                       True, False)


(3253, 2)

# Examples of joins

To make more realistic of rows dropping, we're going to work only with ownership_type == LP

In [20]:
sd_active_biz.ownership_type.value_counts()

SOLE      28719
CORP      18569
LLC        7912
SCORP      4555
H-W        1959
PARTNR     1671
NO/PRF      707
LP          466
TRUST        13
Name: ownership_type, dtype: int64

In [21]:
sd_lp = sd_active_biz[sd_active_biz.ownership_type == "LP"].copy()

"""The full SD data has {full_unique} naics codes while LPs come from {lp_unique} codes and \
there are {cen_unique} naics codes \
in this census file""".format(full_unique = len(sd_active_biz.naics_sector.unique()),
            lp_unique = len(sd_lp.naics_sector.unique()),
            cen_unique = len(naics_all['2017naicscode'][naics_all.is_twodig_code].unique()))



'The full SD data has 24 naics codes while LPs come from 22 codes and there are 17 naics codes in this census file'

In [22]:

## final check on dtypes
if sd_lp['naics_sector'].dtypes != naics_all['2017naicscode'].dtypes:
    naics_all['2017naicscode'] = naics_all['2017naicscode'].astype(str)
    sd_lp['naics_sector'] = sd_lp['naics_sector'].astype(str)

## "Inner join"- retain only two-dig naics codes in both 

- Filter to two-digit naics codes in the Census data
- Inner join the sd_lp data and these two-digit codes

After the merge, print diagnostics on dropped rows in SD businesses data and dropped NAICS codes

In [28]:
naics_twodig = naics_all[naics_all.is_twodig_code].copy()

In [29]:
sd_cen_inner = pd.merge(sd_lp,
                       naics_twodig,
                       how = "inner",
                       left_on = "naics_sector",
                       right_on = "2017naicscode")

"""In our original data, there were {n_orig} rows; \
now after dropping ones without a naics code \
in our current census data there are {n_new} rows\
""".format(n_orig = sd_lp.shape[0], 
          n_new = sd_cen_inner.shape[0])

## which naics codes got lost
## subset to account keys not in the 
## inner join and to the naics columns
lost_merge = sd_lp.loc[~sd_lp.account_key.isin(sd_cen_inner.account_key),
                      ['naics_sector', 'naics_description']].drop_duplicates()

lost_merge

## see ones like cpa; can also aggregate; seems at least some are misc
## and are probably in census data full 6-dig naics codes
lost_merge_dx = lost_merge.groupby('naics_sector').agg({'naics_description': lambda x: "; ".join(x)})

lost_merge_dx


'In our original data, there were 466 rows; now after dropping ones without a naics code in our current census data there are 403 rows'

,naics_sector,naics_description
4099,48,MOTOR VEHICLE TOWING
5855,44,GROCERY STORES
6563,49,WAREHOUSING & STORAGE
7489,32,ALL OTHER MISCELLANEOUS WOOD PRODUCT MFG
10796,44,GASOLINE STATIONS
13945,45,ALL OTHER MISC STORE RETAILERS (EXC TOBACCO)
15177,33,ALL OTH ELECTRICAL EQUIPMENT & COMPONENT MFG
15573,45,MISCELLANEOUS STORE RETAILERS
17179,44,USED CAR DEALERS
19656,44,GASOLINE STATIONS WITH CONVENIENCE STORES


,naics_description
naics_sector,
31,COFFEE & TEA MFG; WINERIES; ALL OTHER MISCELLA...
32,ALL OTHER MISCELLANEOUS WOOD PRODUCT MFG; COMM...
33,ALL OTH ELECTRICAL EQUIPMENT & COMPONENT MFG; ...
44,GROCERY STORES; GASOLINE STATIONS; USED CAR DE...
45,ALL OTHER MISC STORE RETAILERS (EXC TOBACCO); ...
48,MOTOR VEHICLE TOWING; OTHER AIR TRANSPORTATION...
49,WAREHOUSING & STORAGE; LOCAL MESSENGERS & LOCA...


## "Left join"- retain all sd businesses even if 2-dig naics code not in census

In [31]:
naics_twodig.head()

,2017naicscode,2017naicstitle,is_twodig_code
1,11,"Agriculture, Forestry, Fishing and Hunting",True
132,21,"Mining, Quarrying, and Oil and Gas Extraction",True
180,22,Utilities,True
205,23,Construction,True
926,42,Wholesale Trade,True


In [34]:
## try merging 
sd_licensed_wnaics = pd.merge(sd_lp,
                             naics_twodig,
                             left_on = 'naics_sector',
                             right_on = '2017naicscode',
                             how = "left",
                             indicator = "naics_merge_status")
sd_licensed_wnaics.naics_merge_status.value_counts()

## look at sample of ones that didnt merge

sd_licensed_wnaics.loc[sd_licensed_wnaics.naics_merge_status == "left_only",
                  ['naics_code', 'naics_description', '2017naicstitle',
                  'dba_name']].sample(20)


both          403
left_only      63
right_only      0
Name: naics_merge_status, dtype: int64

,naics_code,naics_description,2017naicstitle,dba_name
375,48819,OTHER AIR TRANSPORTATION SUPPORT ACTIVITIES,NaN,COAST AVIATION SERVICES LLC
331,4431,ELECTRONICS & APPLIANCE STORES,NaN,SPRINT 8943
445,4539,OTHER MISCELLANEOUS STORE RETAILERS,NaN,JOHNSON CONTROLS FIRE PROTECTION LP
404,454,NONSTORE RETAILERS,NaN,VORFREUDE CAPITAL
110,44711,GASOLINE STATIONS WITH CONVENIENCE STORES,NaN,4S RANCH GASOLINE & CAR WASH
212,44531,"BEER, WINE & LIQUOR STORES",NaN,HKG DUTY FREE
332,443,ELECTRONICS & APPLIANCE STORES,NaN,SPRINT 8934
399,44112,USED CAR DEALERS,NaN,ODAI AUTO SALES
371,45439,OTHER DIRECT SELLING ESTABLISHMENTS,NaN,PRIME HARVEST LLC
373,45322,"GIFT, NOVELTY & SOUVENIR STORES",NaN,AR WORKSHOP SAN DIEGO


# Activity

- Going back to the full sd_active_biz data
- Go back to the six-digit NAICS codes and try the following merges:
    
    - Inner join with census data
    - Right join where in the sd_active_biz data, find the # of businesses per NAICS code and then merge that with the Census data
    - Left join with census data retaining all san diego businesses

    
- Using other fields in the SD businesses data like the date the business was started, do some merge diagnostics of whether, for instance, older businesses are more or less likely to be lost in the left join